In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
import sys
sys.path.append('../')

In [2]:
pl = pd.read_csv('./data/openpowerlifting.csv',encoding="ISO-8859-1")
m=pd.read_csv('./data/meets.csv',encoding="ISO-8859-1")

In [3]:
pl.dropna(axis=0, how = "all",inplace=True)
pl.dropna(axis=1, how = "all",inplace=True)

In [4]:
# Calculates the percentage of null registers for each variable

percent_missing = round(pl.isnull().sum() * 100 / len(pl), 2)

In [5]:
percent_missing.items

<bound method Series.items of MeetID             0.00
Name               0.00
Sex                0.00
Equipment          0.00
Age               61.94
Division           4.08
BodyweightKg       0.62
WeightClassKg      1.00
Squat4Kg          99.68
BestSquatKg       22.89
Bench4Kg          99.49
BestBenchKg        7.78
Deadlift4Kg       99.27
BestDeadliftKg    17.73
TotalKg            5.98
Place              0.28
Wilks              6.25
dtype: float64>

In [6]:
for c,n in zip(percent_missing.index,percent_missing.values):
    if n>70:
        pl.drop([c], axis=1, inplace=True)

In [7]:
pl_d=pl["Division"]

In [8]:
pl.drop(["Division"], axis=1,inplace=True)
pl.shape

(390526, 13)

In [9]:
pl.drop_duplicates(inplace=True)

In [10]:
pl.shape

(381117, 13)

In [11]:
pl = pl.merge(pl_d, left_index=True, right_index=True)
pl.shape


(381117, 14)

In [12]:
    m_d=m[["MeetID","Date"]]

In [13]:
pl_f = pl.merge(m_d, on="MeetID")
pl_f.shape

(381117, 15)

In [14]:
col_n = list(pl_f.select_dtypes(include="object").columns) 
for col in list(pl_f.columns):
    if col in col_n:
        pl_f.fillna("UNK",inplace=True)
    else:
        pl_f.fillna(0,inplace=True)

In [15]:
pl_f.isnull().sum()

MeetID            0
Name              0
Sex               0
Equipment         0
Age               0
BodyweightKg      0
WeightClassKg     0
BestSquatKg       0
BestBenchKg       0
BestDeadliftKg    0
TotalKg           0
Place             0
Wilks             0
Division          0
Date              0
dtype: int64

In [16]:
pl_f

,MeetID,Name,Sex,Equipment,Age,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,Division,Date
0,0,Angie Belk Terry,F,Wraps,47.0,59.60,60,47.63,20.41,70.31,138.35,1,155.05,Mst 45-49,10/29/16
1,0,Dawn Bogart,F,Single-ply,42.0,58.51,60,142.88,95.25,163.29,401.42,1,456.38,Mst 40-44,10/29/16
2,0,Dawn Bogart,F,Raw,42.0,58.51,60,0.00,95.25,0.00,95.25,1,108.29,Open Senior,10/29/16
3,0,Destiny Dula,F,Raw,18.0,63.68,67.5,0.00,31.75,90.72,122.47,1,130.47,Teen 18-19,10/29/16
4,0,Courtney Norris,F,Wraps,28.0,62.41,67.5,170.10,77.11,145.15,392.36,1,424.40,Open Senior,10/29/16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381112,8552,William Barabas,M,Multi-ply,0.0,113.58,125,0.00,0.00,347.50,347.50,2,202.60,Elite,3/3/17
381113,8552,Justin Zottl,M,Multi-ply,0.0,119.02,125,0.00,0.00,322.50,322.50,3,185.77,Elite,3/3/17
381114,8552,Jake Anderson,M,Multi-ply,0.0,120.29,125,0.00,0.00,367.50,367.50,1,211.17,Elite,3/3/17
381115,8552,Jeff Bumanglag,M,Multi-ply,0.0,126.73,140,0.00,0.00,320.00,320.00,3,181.85,Elite,3/3/17


In [17]:
pl_f["Squat%"]= round(pl_f.BestSquatKg / pl_f.BodyweightKg,3)
pl_f["Bench%"]= round(pl_f.BestBenchKg / pl_f.BodyweightKg,3)
pl_f["Dead%"]= round(pl_f.BestDeadliftKg / pl_f.BodyweightKg,3)
pl_f["Total%"]= round(pl_f.TotalKg / pl_f.BodyweightKg,3)

In [18]:
pl_f

,MeetID,Name,Sex,Equipment,Age,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,Division,Date,Squat%,Bench%,Dead%,Total%
0,0,Angie Belk Terry,F,Wraps,47.0,59.60,60,47.63,20.41,70.31,138.35,1,155.05,Mst 45-49,10/29/16,0.799,0.342,1.180,2.321
1,0,Dawn Bogart,F,Single-ply,42.0,58.51,60,142.88,95.25,163.29,401.42,1,456.38,Mst 40-44,10/29/16,2.442,1.628,2.791,6.861
2,0,Dawn Bogart,F,Raw,42.0,58.51,60,0.00,95.25,0.00,95.25,1,108.29,Open Senior,10/29/16,0.000,1.628,0.000,1.628
3,0,Destiny Dula,F,Raw,18.0,63.68,67.5,0.00,31.75,90.72,122.47,1,130.47,Teen 18-19,10/29/16,0.000,0.499,1.425,1.923
4,0,Courtney Norris,F,Wraps,28.0,62.41,67.5,170.10,77.11,145.15,392.36,1,424.40,Open Senior,10/29/16,2.726,1.236,2.326,6.287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381112,8552,William Barabas,M,Multi-ply,0.0,113.58,125,0.00,0.00,347.50,347.50,2,202.60,Elite,3/3/17,0.000,0.000,3.060,3.060
381113,8552,Justin Zottl,M,Multi-ply,0.0,119.02,125,0.00,0.00,322.50,322.50,3,185.77,Elite,3/3/17,0.000,0.000,2.710,2.710
381114,8552,Jake Anderson,M,Multi-ply,0.0,120.29,125,0.00,0.00,367.50,367.50,1,211.17,Elite,3/3/17,0.000,0.000,3.055,3.055
381115,8552,Jeff Bumanglag,M,Multi-ply,0.0,126.73,140,0.00,0.00,320.00,320.00,3,181.85,Elite,3/3/17,0.000,0.000,2.525,2.525


In [19]:
for col in list(pl_f.select_dtypes(include=["int64", "float64"]).columns) :
        pl_f[col] = pl_f[col].apply(lambda x: 0 if x<0 else x)

In [20]:
pl_f.to_csv ('./output/pl.csv', index=False)